In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Approaching the problem
Decision Tree is almost identical to when we want to decide a thing. For example, if we want to go swimming, we might consider if it's raining or not, or is the pool indoor or outdoor, etc.. However, computer obviously can't think about the question to ask in order to decide the outcome.

That's why we need a criterion to decide what is the best question to ask. GINI impurity score will be implemented here. This is called choosing the best split.

Like most tree building algorithm, we can build the tree recursively and for the base case, there are many complicated arguments involved, such as max depth of a tree, the min number of sample in a leaf, etc.. But those won't be considered and we will build the tree until no more leaf can be created.

First, let's implement Gini

In [ ]:
def gini(data):
        